## Guardrails

### Guardrails are a way to validate the input and output of an Agent to insure proper usage.

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [3]:
from agents import Agent, Runner, RunContextWrapper, input_guardrail, GuardrailFunctionOutput, TResponseInputItem
from pydantic import BaseModel

In [4]:
class HomeworkCheatDetectionOutput(BaseModel):
    attempting_cheat: bool
    explanation: str

homework_cheat_guardrail_agent = Agent(
    name="Homework Cheat Detector",
    instructions=(
        "Determine if the user's query resembles a typical homework assignment or exam question, indicating an attempt to cheat. General questions about concepts are acceptable. "
        " Cheating: 'Fill in the blank: The capital of France is ____.',"
        " 'Which of the following best describes photosynthesis? A) Cellular respiration B) Conversion of light energy C) Evaporation D) Fermentation.'"
        " Not-Cheating: 'What is the capital of France?', 'Explain photosynthesis.'"
    ),
    output_type= HomeworkCheatDetectionOutput,
    model= "gpt-4o-mini"
)

@input_guardrail
async def cheat_detection_guardrails(ctx: RunContextWrapper[None], agent: Agent, 
                                     input: str | list[TResponseInputItem]) -> GuardrailFunctionOutput:
    detection_result = await Runner.run(homework_cheat_guardrail_agent, input)

    return GuardrailFunctionOutput(
        tripwire_triggered=detection_result.final_output.attempting_cheat,
        output_info=detection_result.final_output
    )

study_helper_agent = Agent(
    name = "Study Helper Agent",
    instructions= "You assist users in studying by explaining concepts or providing guidance, without directly solving homework or test questions.",
    model= "gpt-4o",
    input_guardrails=[cheat_detection_guardrails]
)

In [5]:
# This should trigger the cheat detection guardrail
from agents import InputGuardrailTripwireTriggered


try:
    response = await Runner.run(study_helper_agent, "Fill in the blank: The process of converting light energy into chemical energy is called ____.")
    print("Guardrail didn't trigger")
    print("Response: ", response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Homework cheat guardrail triggered")
    print("Exception details:", str(e))

Homework cheat guardrail triggered
Exception details: Guardrail InputGuardrail triggered tripwire


In [7]:
# This should trigger the cheat detection guardrail
try:
    response = await Runner.run(study_helper_agent, "What is the process of converting light energy into chemical energy?")
    print("Guardrail didn't trigger")
    print("Response: ", response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Homework cheat guardrail triggered")
    print("Exception details:", str(e))

Guardrail didn't trigger
Response:  The process of converting light energy into chemical energy is known as photosynthesis. This process primarily occurs in plants, algae, and certain bacteria. Here's a brief overview of how it works:

1. **Light Absorption**: Chlorophyll, the green pigment in chloroplasts, absorbs sunlight. This energy excites electrons, starting the photosynthesis process.

2. **Water Splitting (Photolysis)**: The absorbed light energy splits water molecules (H₂O) into oxygen (O₂), protons, and electrons. This process occurs in the thylakoid membranes of the chloroplasts.

3. **Electron Transport Chain**: The excited electrons move through a series of proteins embedded in the thylakoid membrane, known as the electron transport chain. As electrons pass through these proteins, they lose energy, which is used to pump protons across the membrane, creating a proton gradient.

4. **ATP and NADPH Formation**: The proton gradient drives the production of ATP (adenosine triph

In [8]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    output_guardrail,
)

class MessageOutput(BaseModel):
    response: str

@output_guardrail
async def forbidden_words_guardrail(ctx: RunContextWrapper, agent: Agent, output: str) -> GuardrailFunctionOutput:
    print(f"Checking output for forbidden phrases: {output}")

    # Funny forbidden phrases to check
    forbidden_phrases = ["fart", "booger", "silly goose"]

    # Convert output to lowercase for case-insensitive comparison
    output_lower = output.lower()

    # Check which forbidden phrases are present in the response
    found_phrases = [phrase for phrase in forbidden_phrases if phrase in output_lower]
    trip_triggered = bool(found_phrases)

    print(f"Found forbidden phrases: {found_phrases}")

    return GuardrailFunctionOutput(
        output_info={
            "reason": "Output contains forbidden phrases.",
            "forbidden_phrases_found": found_phrases,
        },
        tripwire_triggered=trip_triggered,
    )

agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[forbidden_words_guardrail],
    model="gpt-4o-mini",
)

In [9]:
try:
    await Runner.run(agent, "Say the word fart")
    print("Guardrail didn't trip - this is unexpected")
except OutputGuardrailTripwireTriggered:
    print("The agent said a bad word, he is fired.")

Checking output for forbidden phrases: Fart! How can I assist you today?
Found forbidden phrases: ['fart']
The agent said a bad word, he is fired.


In [10]:
try:
    await Runner.run(agent, "Hey wassup")
    print("Guardrail didn't trip yay")
except OutputGuardrailTripwireTriggered:
    print("The agent said a bad word, he is fired.")

Checking output for forbidden phrases: Hello! I'm here to help. What can I assist you with today?
Found forbidden phrases: []
Guardrail didn't trip yay
